In [1]:
# -*- coding: utf-8 -*-
# @Time    : 2021/4/10 18:08
# @Author  : Weiming Mai
# @FileName: main.py
# @Software: PyCharm

import numpy as np
from Nodes import *
from Utils import *
from numpy import *
import xlrd
from sklearn import preprocessing

In [2]:
# read data
def excel_to_matrix(path, num):
    table = xlrd.open_workbook(path).sheets()[num]  # 获取第一个sheet表
    row = table.nrows  # 行数
    col = table.ncols  # 列数
    datamatrix = np.zeros((row, col))  # 生成一个nrows行ncols列，且元素均为0的初始矩阵
    for x in range(col):
        cols = np.matrix(table.col_values(x))  # 把list转换为矩阵进行矩阵操作
        datamatrix[:, x] = cols  # 按列把数据存进矩阵中
        # 数据归一化
        min_max_scaler = preprocessing.MinMaxScaler()
        datamatrix = min_max_scaler.fit_transform(datamatrix)
    return datamatrix

def mini_batch(batch_size, data_X, data_Y):
    idx = np.arange(0, data_X.shape[0])
    np.random.shuffle(idx)
    idx = idx[0:batch_size]

    return data_X[idx,:], data_Y[idx,:]

datafile = "C:/Users/13271/Desktop/git/DL/iris_test+train.xlsx"

data_x = excel_to_matrix(datafile, 0)
data_y = excel_to_matrix(datafile, 3)

x_test = excel_to_matrix(datafile, 1)
y_test = excel_to_matrix(datafile, 2)

In [3]:
batch_size = 32
x_batch, y_batch = mini_batch(batch_size, data_x, data_y)

x = Input(x_batch)
l2 = Dense(x, 6)
# l2 = Sigmoid(l2)
l3 = Dense(l2, 4)
# l3 = Sigmoid(l3)
l4 = Dense(l3, 4)
# l4 = Sigmoid(l4)
l5 = Dense(l4, 3)
# l5 = Sigmoid(l5)


y = Const(y_batch)
cost = MSE(l5, y)

Forward(cost)
p = Backprop(cost)
GD = G_D_Optimizer(p, 0.005) # lr
print(cost.value)
epoch = 1000
for ep in range(epoch):
    x_batch, y_batch = mini_batch(batch_size, data_x,data_y)
    x.value = x_batch
    y.value = y_batch
    for j in range(20):
        Forward(cost)
        Backprop(cost)
        GD.train()
    if ep % 100 == 0:
        print(cost.value)
#
#test = preprocessing.MinMaxScaler().fit_transform(np.array([[5.1,3.4,1.5,0.2]]))
# x.value = np.array([[5.1,3.4,1.5,0.2]])
# x.value = test
x.value = x_test
Forward(cost)
# Backprop(cost)
print(cost.value)
print(l5.value)

1.1081623319125744
0.6619803915781409
0.23430378013100348
0.2105923267776595
0.24104673591461737
0.16673921037448197
0.30783438098942395
0.2671147918731628
0.2883287962891622
0.29619280720451874
0.2483742629483443
1.0653659646137814
[[ 1.10154793  0.03979461 -0.1414362 ]
 [ 1.17712427 -0.09897933 -0.07820734]
 [ 0.61324372  0.77605362 -0.3889271 ]
 [ 1.03045235  0.06258759 -0.09301044]
 [ 1.0745694  -0.1800865   0.10549347]
 [ 1.16980922 -0.32865861  0.15860852]
 [ 0.91983815  0.27132152 -0.19105883]
 [ 1.26081761 -0.27874042  0.01767286]
 [ 1.01507629  0.1039536  -0.11902285]
 [ 1.24524148 -0.16500885 -0.08043658]
 [ 1.07507817  0.09770704 -0.17282437]
 [ 0.01380608  0.60102138  0.38528856]
 [-0.01972435  0.56815131  0.45155731]
 [ 0.11560116  0.30302985  0.58122262]
 [ 0.08172322  0.60735742  0.31096229]
 [ 0.09780476  0.73382777  0.16859577]
 [ 0.0631144   0.46659013  0.47030533]
 [ 0.21472321  0.27436927  0.51078021]
 [ 0.15769969  0.32385236  0.51838522]
 [ 0.15321726  0.42103751 

In [19]:
def activation_function(choice, x):  # activation function ,output vector or num, x is a matrices
    if choice == 1:  # sigmoid

        return 1.0 / (1 + np.exp(-x))
    else:
        return np.maximum(x, 0.0)  # relu
        # return abs(x)


def function_derivative(choice, x):
    derivative = np.zeros([1, x.shape[1]])
    if choice == 1:
        return np.multiply((1 / (1 + np.exp(-x))), (1 - 1 / (1 + np.exp(-x))))  # 此处用点乘 * 不行，要用multiply       sigmoid
    else:
        for i in range(x.shape[1]):
            if x[0, i] > 0:
                derivative[0, i] = 1
            else:
                derivative[0, i] = 0
        return derivative


class NeuralNetwork:
    def __init__(self, x, y, alpha=0.01):  # 输入矩阵行都是行向量属性，列是样本数
        self.input = x
        self.y = y
        self.hidden_neuron_num = 4
        self.choice = 2  # 1:sigmoid, 2:relu,第一层用choice，第二层choice2
        self.choice2 = 1
        self.weights1 = np.random.rand(self.input.shape[1],
                                       self.hidden_neuron_num)  # 1*4     #initialize layer1 weights,hidden_neutal == 4 ,是矩阵
        # self.weights1 = np.random.uniform(1,50,(1,4))
        self.weights2 = np.random.rand(self.hidden_neuron_num, self.y.shape[1])  # 4*1
        # self.weights2 = np.random.uniform(1,50,(4,1))
        self.bias1 = np.random.rand(1, 1)
        # self.bias1 = np.array([[0.1]])
        self.bias2 = np.random.rand(1, 1)
        # self.bias2 = np.array([[0.1]])
        self.output = np.zeros(self.y.shape[1])  # initialize output
        self.learning_rate = alpha
        self.loss = 0

    def feedforward(self, current_x):
        self.layer1 = activation_function(self.choice, np.dot(current_x, self.weights1) + self.bias1 * np.ones(
            self.hidden_neuron_num))  # current_x:1*1,self.weights:1*4,hidden_neural == 4
        self.output = activation_function(self.choice2,
                                          np.dot(self.layer1, self.weights2) + self.bias2 * np.ones(self.y.shape[1]))

    def backpropagation(self, current_x, current_y):
        d_weights2 = np.dot(self.layer1.T, np.multiply(2 * (self.output - mat(current_y)),
                                                       function_derivative(self.choice2,
                                                                           self.output)))  # 这里的 *是点乘，适合输出为多维情况
        d_weights1 = np.dot(mat(current_x).T, np.multiply(
            np.dot(np.multiply(function_derivative(self.choice2, self.output), 2 * (self.output - mat(current_y))),
                   self.weights2.T), function_derivative(self.choice, self.layer1)))
        d_bias2 = np.dot(2 * (self.output - mat(current_y)), function_derivative(self.choice, self.output).T)
        d_bias1 = np.dot(
            np.dot(np.multiply(function_derivative(self.choice, self.output), 2 * (self.output - mat(current_y))),
                   self.weights2.T), function_derivative(self.choice, self.layer1).T)

        # update weights and bias
        self.weights1 -= self.learning_rate * d_weights1
        self.weights2 -= self.learning_rate * d_weights2
        self.bias1 -= self.learning_rate * d_bias1
        self.bias2 -= self.learning_rate * d_bias2

    # def train(self):
    #     #         batch = 30
    #     #         i = random.randint(0,120)
    #     for i in range(self.input.shape[0]):
    #         self.feedforward(self.input[i])
    #         self.backpropagation(self.input[i], self.y[i])
    #         #         for i in range(self.input.shape[0]):
    #         #             self.feedforward(self.input[i])
    #         self.loss += np.sum(np.square(self.output - self.y[i])) / self.input.shape[0]

    def train(self):

        for i in range(self.input.shape[0]):
            self.feedforward(self.input[i])
            self.backpropagation(self.input[i], self.y[i])
        for j in range(self.input.shape[0]):
            self.feedforward(self.input[j])
            self.loss += np.sum(np.square(self.output - self.y[j])) / self.input.shape[0]

    def test(self, test_x, test_y):
        prediction = np.zeros((test_y.shape[0], test_y.shape[1]))
        for i in range(test_x.shape[0]):
            self.feedforward(test_x[i])
            prediction[i] = self.output[0]
        print(prediction)


if __name__ == "__main__":
    # x = np.linspace(-1,1,num = 300)[:,np.newaxis]
    # noise = np.random.normal(0, 0.05, x.shape).astype(np.float32)
    # y = np.square(x) - 0.5 + noise

    #     x = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1]])
    #     y = np.array([[0],[1],[1],[0]])

    #     x_test = np.array([[0,0,1],[0,1,1],[1,0,1],[1,1,1],[0, 1, 1],[[1, 1, 0]]])
    #     y_test = np.array([[0],[1],[1],[0],[1],[0]])
    # x = np.array([[0,0],[1,0],[1,1],[0,1]])
    # y = np.array([[0],[0],[1],[0]])

    NN = NeuralNetwork(data_x, data_y)
    for j in range(1500):
        NN.loss = 0
        NN.train()

        if j % 50 == 0:
            print(NN.loss)

0.6671848785796811
0.5750518878054989
0.6311979863149375
0.6251713211083346
0.5497766531106618
0.467578016527202
0.41063521828971916
0.3772833736570787
0.355216224592068
0.3409338205110204
0.33117208215204097
0.3241953154268692
0.31963446159142406
0.316259545971314
0.31378652669672497
0.31195329307644976
0.31054097889955284
0.30956193292426726
0.30874693841406
0.3081423369475739
0.30766886141573047
0.30729748143627234
0.3070044701484074
0.3067709983709008
0.3065801113675294
0.3064200394154444
0.3062800722364157
0.3061506537624774
0.306023580804198
0.30589188954856505


In [21]:
NN.test(x_test, y_test)

[[3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [3.37534711e-01 3.37534711e-01 3.37534711e-01]
 [5.40836029e-07 9.56945771e-01 3.81545370e-02]
 [2.06345815e-07 6.88860237e-01 2.58818671e-01]
 [1.24808778e-10 9.63484494e-01 4.40104829e-02]
 [1.22575978e-07 9.99880293e-01 1.60035337e-04]
 [5.11474666e-03 9.20558062e-01 3.64223810e-02]
 [7.60326316e-08 9.71038636e-01 2.78352190e-02]
 [3.68772455e-08 9.99937634e-01 9.14787560e-05]
 [2.22412927e-08 9.98914344e-01 1.32277098e-03]
 [5.62743748e-10 9.99919730e-01 1.35308832e-04]
 [2.31911210e-03 9.52097631e-01 2.355262

In [4]:
np.random.rand(3,4)

array([[0.60346339, 0.55294892, 0.56598759, 0.00463989],
       [0.29861183, 0.61777568, 0.96514733, 0.00568809],
       [0.90294718, 0.57296699, 0.29898411, 0.82365922]])